In [1]:
# TO DO
#
# MVP: create a version that allows us to pass and no more

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Copied from assignment 2. NEEDS WORK!!!

In [2]:
# Organize data for pytorch DataLoader
import os
import shutil
import random
import torch

DATA_DIR = '/Users/hartih/Documents/School/Deep learning/Final_project/dl2021-image-corpus-proj/'
ANNOTATIONS_DIR = DATA_DIR + 'annotations/'
IMAGES_DIR = DATA_DIR + 'images/'

# New fodlers for train, test, and dev sets
TRAIN_DIR = DATA_DIR + 'train/'
DEV_DIR = DATA_DIR + 'dev/'
TEST_DIR = DATA_DIR + 'test/'

annotations = ["baby",
               "bird",
               "car",
               "clouds",
               "dog",
               "female",
               "flower",
               "male",
               "night",
               "people",
               "portrait",
               "river",
               "sea",
               "tree"]

/Users/hartih/opt/anaconda3/envs/deep_learning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create labels for images
image_file_names = os.listdir(IMAGES_DIR)
dict_labels = {}
for image_file_name in image_file_names:  # Initiate label tensors
    if os.path.isfile(IMAGES_DIR + image_file_name):
        dict_labels[image_file_name] = torch.zeros(14)
for i in range(len(annotations)):  # Fill label tensors with 1's if found in one of the annotations text files
    with open(ANNOTATIONS_DIR + annotations[i] + ".txt") as f:
        for row in f:
            row = "im" + row.strip() + ".jpg"
            dict_labels[row][i] = 1

In [4]:
dict_labels

{'im1976.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im12710.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im14361.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im17668.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.]),
 'im10107.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.]),
 'im11219.jpg': tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im19645.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im16576.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.]),
 'im19123.jpg': tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.]),
 'im16210.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 'im10661.jpg': tensor([0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0.]),
 'im3807.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im8952.jpg': ten

In [5]:
# Split the data to train, test, and dev
os.makedirs(TRAIN_DIR)
os.makedirs(TEST_DIR)
os.makedirs(DEV_DIR)
for image_file_name in image_file_names:
    if os.path.isfile(IMAGES_DIR + image_file_name):
        division = random.randint(1, 3)
        if division == 1:
            shutil.copyfile(IMAGES_DIR + image_file_name, TRAIN_DIR + image_file_name)
        if division == 2:
            shutil.copyfile(IMAGES_DIR + image_file_name, DEV_DIR + image_file_name)
        if division == 3:
            shutil.copyfile(IMAGES_DIR + image_file_name, TEST_DIR + image_file_name)

FileExistsError: [Errno 17] File exists: '/Users/hartih/Documents/School/Deep learning/Final_project/dl2021-image-corpus-proj/train/'

In [6]:
print( len(os.listdir(TEST_DIR)) + len(os.listdir(TRAIN_DIR)) + len( os.listdir(DEV_DIR)) == len(os.listdir(IMAGES_DIR)) )
print(len(os.listdir(TEST_DIR)))
print(len(os.listdir(TRAIN_DIR)))
print(len(os.listdir(DEV_DIR)))
print(len(os.listdir(IMAGES_DIR)))


True
6571
6623
6806
20000


In [7]:
import torch.utils.data
from PIL import Image

# Enable creating train, test, and dev test datasets for PyTorch
class myDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir=IMAGES_DIR, transform=None):
        self.transform = transform
        self.root_dir = root_dir
        self.images = [root_dir + img for img in os.listdir(root_dir)]                
      
    def __len__(self):
        return len(self.images)       

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = img = Image.open(img_path)     
       
        if self.transform:
            img = self.transform(img)     

        return img, dict_labels[img_path.split("/")[-1]]

In [8]:
from torchvision import transforms

train_transform = transforms.Compose([
                                        #transforms.RandomHorizontalFlip(),
                                        transforms.Grayscale(num_output_channels=3),
                                        transforms.ToTensor()])
test_transform = transforms.Compose([
                                        transforms.Grayscale(num_output_channels=3),
                                        transforms.ToTensor()])

# Create datasets for CNN
train_set = myDataset(TRAIN_DIR, transform=train_transform)
test_set = myDataset(TEST_DIR, transform=test_transform)
dev_set = myDataset(DEV_DIR, transform=test_transform)

In [9]:
import torch 

# Calculate the amount of correctly predicted as well as total predictions done
def calc_correct(pred: torch.Tensor, target: torch.Tensor):
    pred = torch.sigmoid(pred)  # Since our neural network does not apply sigmoid
    correct_dict = {'tot': [0,0]}  # First number in value is correct ones, the second one is total amount
    correct_dict['tot_strict'] = [0,0] # All correct
    for i in range(len(pred)): # [100,14]
        all_correct = 0
        total = 0
        # estim_pred_array = []
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            # estim_pred_array.append(estim_pred)
            if annotations[j] not in correct_dict.keys():
                correct_dict[annotations[j]] = [0,0]
            correct_dict['tot'][1] += 1
            correct_dict[annotations[j]][1] += 1
            correct_dict['tot'][0] += int(estim_pred == target[i][j])
            correct_dict[annotations[j]][0] += int(estim_pred == target[i][j])
            all_correct += int(estim_pred == target[i][j])
            total += 1
        correct_dict['tot_strict'][1] += 1
        correct_dict['tot_strict'][0] += int(total==all_correct)
    # To-do how many pictures were entirely correct (accuracy)
    return correct_dict


In [10]:
def class_evaluation(pred: torch.Tensor, target: torch.Tensor):
    pred = torch.sigmoid(pred)
    
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    negative = 0
    positive = 0
    
    for i in range(len(pred)): # [100,14]
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            
            # Negative target values
            if target[i][j] == 0:
                negative += 1
                if estim_pred == 1:
                    false_positive += 1
                if estim_pred == 0:
                    true_negative += 1
            
            # Positive target values
            if target[i][j] == 1:
                positive += 1
                if estim_pred == 1:
                    true_positive += 1
                if estim_pred == 0:
                    false_negative += 1
    
    result = {"true_positive": true_positive,
            "false_positive": false_positive,
            "true_negative": true_negative,
            "false_negative": false_negative,
            "negative": negative,
            "positive": positive}
                    
    return result

In [11]:
def class_evaluation_by_annotation(pred: torch.Tensor, target: torch.Tensor):
    
    pred = torch.sigmoid(pred)
    
    # Initiate vslues for every annotation
    eval_dict = {}
    for a in range(len(annotations)):
        eval_dict[annotations[a]] = {"true_positive": 0,
                                        "false_positive": 0,
                                        "true_negative": 0,
                                        "false_negative": 0,
                                        "negative": 0,
                                        "positive": 0}
    
    for i in range(len(pred)):
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            
            # Negative target values
            if target[i][j] == 0:
                eval_dict[annotations[j]]["negative"] += 1
                if estim_pred == 1:
                    eval_dict[annotations[j]]["false_positive"] += 1
                if estim_pred == 0:
                    eval_dict[annotations[j]]["true_negative"] += 1
            
            # Positive target values
            if target[i][j] == 1:
                eval_dict[annotations[j]]["positive"] += 1
                if estim_pred == 1:
                    eval_dict[annotations[j]]["true_positive"] += 1
                if estim_pred == 0:
                    eval_dict[annotations[j]]["false_negative"] += 1
                    
    return eval_dict

In [ ]:
import torch
import torch.optim as optim
import torch.utils.data
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math


#--- hyperparameters ---
N_EPOCHS = 2
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_TEST = 100
LR = 0.0003
WEIGHT_DECAY = 0.001
MOMENTUM = 0.1


#--- fixed constants ---
NUM_CLASSES = 14
NUM_CHANNELS = 3



# --- Dataset initialization ---

# We transform image files' contents to tensors
# Plus, we can add random transformations to the training data if we like
# Think on what kind of transformations may be meaningful for this data.
# Eg., horizontal-flip is definitely a bad idea for sign language data.
# You can use another transformation here if you find a better one.


# Create Pytorch data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE_TEST, shuffle=False)
dev_loader = torch.utils.data.DataLoader(dataset=dev_set, shuffle=False)


#--- model ---
class CNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(NUM_CHANNELS, 20, (5,5)),
            nn.ReLU(),
            nn.Dropout(p = 0.15),
            nn.MaxPool2d((2,2), stride = (2,2)),
            nn.Conv2d(20, 50, (5,5)),
            nn.ReLU(),
            nn.Dropout(p = 0.15),
            nn.MaxPool2d((2,2), stride = (2,2)),
            nn.Conv2d(50, 100, (5,5)),
            nn.ReLU(),
            nn.Dropout(p = 0.15),
            nn.MaxPool2d((2,2), stride = (2,2))
        )
        self.flatten = nn.Flatten()
        self.lin = nn.Sequential(
            nn.Linear(14400, 500),
            nn.ReLU(),
            nn.Dropout(p = 0.15),
            nn.Linear(500, 100),
            nn.ReLU(),
            nn.Dropout(p = 0.15),
            nn.Linear(100, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.lin(x)
        return x



#--- set up ---
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = CNN().to(device)

# WRITE CODE HERE
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM)
loss_function = nn.BCEWithLogitsLoss()

dev_loss = math.inf
dev_losses = []
dev_accuracies = []
stop_early = False

#--- training ---
for epoch in range(N_EPOCHS):
    if stop_early:
        break
    train_loss = 0
    train_correct = {annotation: [0,0] for annotation in annotations}
    train_correct['tot'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

    total = 0
    for batch_num, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data)
        loss = loss_function(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += len(data)
        train_loss += loss.item()
        new_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_correct[annotation]
            train_correct[annotation][0] += new[0]
            train_correct[annotation][1] += new[1]
        train_correct['tot'][0] += new_correct['tot'][0]
        train_correct['tot'][1] += new_correct['tot'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]
        
        print("------------------------")
        print('Training: Epoch %d - Batch %d/%d: Loss: %.4f | Train Acc: %.3f%% (%d/%d)' % 
              (epoch+1, batch_num+1, len(train_loader), train_loss / (batch_num + 1), 
               100. * train_correct['tot'][0] / train_correct['tot'][1], train_correct['tot'][0], train_correct['tot'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")
    
    # WRITE CODE HERE
    # Please implement early stopping here.
    # You can try different versions, simplest way is to calculate the dev error and
    # compare this with the previous dev error, stopping if the error has grown.
    cur_dev_loss = 0
    dev_correct = 0
    total = 0
    dev_correct = {annotation: [0,0] for annotation in annotations}
    dev_correct['tot'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

    with torch.no_grad():
        for batch_num, (data, target) in enumerate(dev_loader):
            data, target = data.to(device), target.to(device)
            pred = model(data)
            loss = loss_function(pred, target)

            total += len(data)
            cur_dev_loss += loss.item()
            new_dev_correct = calc_correct(pred, target)
            for annotation in annotations:
                new = new_dev_correct[annotation]
                dev_correct[annotation][0] += new[0]
                dev_correct[annotation][1] += new[1]
            dev_correct['tot'][0] += new_dev_correct['tot'][0]
            dev_correct['tot'][1] += new_dev_correct['tot'][1]
            
            evaluations = class_evaluation(pred, target)
            evaluation["true_positive"] += evaluations["true_positive"]
            evaluation["false_positive"] += evaluations["false_positive"]
            evaluation["true_negative"] += evaluations["true_negative"]
            evaluation["false_negative"] += evaluations["false_negative"]
            evaluation["positive"] += evaluations["positive"]
            evaluation["negative"] += evaluations["negative"]
            

        current_loss = cur_dev_loss / (len(dev_loader) + 1)
        dev_losses.append(current_loss)
        current_accuracy = {annotation: 100. * dev_correct[annotation][0] / dev_correct[annotation][1]  for annotation in annotations}  # Accuracies for all classes 
        current_accuracy['tot'] = 100. * dev_correct['tot'][0] / dev_correct['tot'][1]
        dev_accuracies.append(current_accuracy)

        if current_loss <= dev_loss:
            dev_loss = current_loss
        else:
            stop_early = True
        
        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Dev Acc: %.3f%% (%d/%d)' % 
            (batch_num+1, len(dev_loader), cur_dev_loss / (len(dev_loader) + 1), 
            100. * dev_correct['tot'][0] / dev_correct['tot'][1], dev_correct['tot'][0], dev_correct['tot'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")
        
print(dev_losses)
print(dev_accuracies)

------------------------
Training: Epoch 1 - Batch 1/67: Loss: 0.6803 | Train Acc: 76.143% (1066/1400)
True positive rate: 33.333% (34/102)
False negative rate: 66.667% (68/102)
True negative rate: 79.507% (1032/1298)
False positive rate: 20.493% (266/1298)
------------------------
------------------------
Training: Epoch 1 - Batch 2/67: Loss: 0.6714 | Train Acc: 76.429% (2140/2800)
True positive rate: 35.808% (82/229)
False negative rate: 64.192% (147/229)
True negative rate: 80.047% (2058/2571)
False positive rate: 19.953% (513/2571)
------------------------
------------------------
Training: Epoch 1 - Batch 3/67: Loss: 0.6560 | Train Acc: 78.762% (3308/4200)
True positive rate: 36.890% (121/328)
False negative rate: 63.110% (207/328)
True negative rate: 82.309% (3187/3872)
False positive rate: 17.691% (685/3872)
------------------------
------------------------
Training: Epoch 1 - Batch 4/67: Loss: 0.6332 | Train Acc: 79.357% (4444/5600)
True positive rate: 34.699% (144/415)
False n

------------------------
Training: Epoch 1 - Batch 30/67: Loss: 0.3284 | Train Acc: 89.260% (37489/42000)
True positive rate: 13.882% (418/3011)
False negative rate: 86.118% (2593/3011)
True negative rate: 95.081% (37071/38989)
False positive rate: 4.919% (1918/38989)
------------------------
------------------------
Training: Epoch 1 - Batch 31/67: Loss: 0.3245 | Train Acc: 89.385% (38793/43400)
True positive rate: 13.815% (428/3098)
False negative rate: 86.185% (2670/3098)
True negative rate: 95.194% (38365/40302)
False positive rate: 4.806% (1937/40302)
------------------------
------------------------
Training: Epoch 1 - Batch 32/67: Loss: 0.3218 | Train Acc: 89.467% (40081/44800)
True positive rate: 13.531% (433/3200)
False negative rate: 86.469% (2767/3200)
True negative rate: 95.308% (39648/41600)
False positive rate: 4.692% (1952/41600)
------------------------
------------------------
Training: Epoch 1 - Batch 33/67: Loss: 0.3188 | Train Acc: 89.524% (41360/46200)
True positiv

------------------------
Training: Epoch 1 - Batch 58/67: Loss: 0.2824 | Train Acc: 90.688% (73639/81200)
True positive rate: 9.725% (570/5861)
False negative rate: 90.275% (5291/5861)
True negative rate: 96.987% (73069/75339)
False positive rate: 3.013% (2270/75339)
------------------------
------------------------
Training: Epoch 1 - Batch 59/67: Loss: 0.2812 | Train Acc: 90.728% (74941/82600)
True positive rate: 9.587% (571/5956)
False negative rate: 90.413% (5385/5956)
True negative rate: 97.033% (74370/76644)
False positive rate: 2.967% (2274/76644)
------------------------
------------------------
Training: Epoch 1 - Batch 60/67: Loss: 0.2804 | Train Acc: 90.765% (76243/84000)
True positive rate: 9.451% (572/6052)
False negative rate: 90.549% (5480/6052)
True negative rate: 97.079% (75671/77948)
False positive rate: 2.921% (2277/77948)
------------------------
------------------------
Training: Epoch 1 - Batch 61/67: Loss: 0.2791 | Train Acc: 90.826% (77565/85400)
True positive r

In [20]:
#--- test ---
test_loss = 0
test_correct = {annotation: [0,0] for annotation in annotations}
test_correct['tot'] = [0,0]
evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

evaluation_by_annotation = {}
for a in range(len(annotations)):
    evaluation_by_annotation[annotations[a]] = {"true_positive": 0,
                                                "false_positive": 0,
                                                "true_negative": 0,
                                                "false_negative": 0,
                                                "negative": 0,
                                                "positive": 0}

with torch.no_grad():
    for batch_num, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        # WRITE CODE HERE
        pred = model(data)
        loss = loss_function(pred, target)

        total += len(data)
        test_loss += loss.item()
        new_test_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_test_correct[annotation]
            test_correct[annotation][0] += new[0]
            test_correct[annotation][1] += new[1]
        test_correct['tot'][0] += new_test_correct['tot'][0]
        test_correct['tot'][1] += new_test_correct['tot'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]
        
        evaluations_by_annotation = class_evaluation_by_annotation(pred, target)
        print(evaluation_by_annotation)
        
        for a in range(len(annotations)):
            evaluation_by_annotation[annotations[a]]["true_positive"] += evaluations_by_annotation[annotations[a]]["true_positive"]
            evaluation_by_annotation[annotations[a]]["false_positive"] += evaluations_by_annotation[annotations[a]]["false_positive"]
            evaluation_by_annotation[annotations[a]]["true_negative"] += evaluations_by_annotation[annotations[a]]["true_negative"]
            evaluation_by_annotation[annotations[a]]["false_negative"] += evaluations_by_annotation[annotations[a]]["false_negative"]
            evaluation_by_annotation[annotations[a]]["positive"] += evaluations_by_annotation[annotations[a]]["positive"]
            evaluation_by_annotation[annotations[a]]["negative"] += evaluations_by_annotation[annotations[a]]["negative"]
        
        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Test Acc: %.3f%% (%d/%d)' % 
              (batch_num+1, len(test_loader), test_loss / (batch_num + 1), 
               100. * test_correct['tot'][0] / test_correct['tot'][1], test_correct['tot'][0], test_correct['tot'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'male': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'night': {'true_pos

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 496, 'false_negative': 4, 'negative': 496, 'positive': 4}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 493, 'false_negative': 7, 'negative': 493, 'positive': 7}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 492, 'false_negative': 8, 'negative': 492, 'positive': 8}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 477, 'false_negative': 23, 'negative': 477, 'positive': 23}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 492, 'false_negative': 8, 'negative': 492, 'positive': 8}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 410, 'false_negative': 90, 'negative': 410, 'positive': 90}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 477, 'false_negative': 23, 'negative': 477, 'positive': 23}, 'male': {'true_positive': 0, 'false_positive': 0, 'true_negative': 421, 'false_negative': 79, 'negative': 4

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 894, 'false_negative': 6, 'negative': 894, 'positive': 6}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 887, 'false_negative': 13, 'negative': 887, 'positive': 13}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 886, 'false_negative': 14, 'negative': 886, 'positive': 14}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 845, 'false_negative': 55, 'negative': 845, 'positive': 55}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 880, 'false_negative': 20, 'negative': 880, 'positive': 20}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 738, 'false_negative': 162, 'negative': 738, 'positive': 162}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 855, 'false_negative': 45, 'negative': 855, 'positive': 45}, 'male': {'true_positive': 0, 'false_positive': 1, 'true_negative': 762, 'false_negative': 137, 'neg

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1292, 'false_negative': 8, 'negative': 1292, 'positive': 8}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1279, 'false_negative': 21, 'negative': 1279, 'positive': 21}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1281, 'false_negative': 19, 'negative': 1281, 'positive': 19}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1225, 'false_negative': 75, 'negative': 1225, 'positive': 75}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1273, 'false_negative': 27, 'negative': 1273, 'positive': 27}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1074, 'false_negative': 226, 'negative': 1074, 'positive': 226}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1241, 'false_negative': 59, 'negative': 1241, 'positive': 59}, 'male': {'true_positive': 0, 'false_positive': 1, 'true_negative': 1100, 'false_negat

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1689, 'false_negative': 11, 'negative': 1689, 'positive': 11}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1673, 'false_negative': 27, 'negative': 1673, 'positive': 27}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1674, 'false_negative': 26, 'negative': 1674, 'positive': 26}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1604, 'false_negative': 96, 'negative': 1604, 'positive': 96}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1665, 'false_negative': 35, 'negative': 1665, 'positive': 35}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1409, 'false_negative': 291, 'negative': 1409, 'positive': 291}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1629, 'false_negative': 71, 'negative': 1629, 'positive': 71}, 'male': {'true_positive': 0, 'false_positive': 1, 'true_negative': 1446, 'false_neg

...

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 4872, 'false_negative': 28, 'negative': 4872, 'positive': 28}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 4809, 'false_negative': 91, 'negative': 4809, 'positive': 91}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 4833, 'false_negative': 67, 'negative': 4833, 'positive': 67}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 4637, 'false_negative': 263, 'negative': 4637, 'positive': 263}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 4791, 'false_negative': 109, 'negative': 4791, 'positive': 109}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 4068, 'false_negative': 832, 'negative': 4068, 'positive': 832}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 4704, 'false_negative': 196, 'negative': 4704, 'positive': 196}, 'male': {'true_positive': 0, 'false_positive': 3, 'true_negative': 4144, 'fal

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5271, 'false_negative': 29, 'negative': 5271, 'positive': 29}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5206, 'false_negative': 94, 'negative': 5206, 'positive': 94}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5227, 'false_negative': 73, 'negative': 5227, 'positive': 73}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5022, 'false_negative': 278, 'negative': 5022, 'positive': 278}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5185, 'false_negative': 115, 'negative': 5185, 'positive': 115}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 4401, 'false_negative': 899, 'negative': 4401, 'positive': 899}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5087, 'false_negative': 213, 'negative': 5087, 'positive': 213}, 'male': {'true_positive': 0, 'false_positive': 3, 'true_negative': 4482, 'fal

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5671, 'false_negative': 29, 'negative': 5671, 'positive': 29}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5603, 'false_negative': 97, 'negative': 5603, 'positive': 97}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5621, 'false_negative': 79, 'negative': 5621, 'positive': 79}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5399, 'false_negative': 301, 'negative': 5399, 'positive': 301}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5573, 'false_negative': 127, 'negative': 5573, 'positive': 127}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 4735, 'false_negative': 965, 'negative': 4735, 'positive': 965}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5474, 'false_negative': 226, 'negative': 5474, 'positive': 226}, 'male': {'true_positive': 0, 'false_positive': 3, 'true_negative': 4812, 'fal

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 6069, 'false_negative': 31, 'negative': 6069, 'positive': 31}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5996, 'false_negative': 104, 'negative': 5996, 'positive': 104}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 6011, 'false_negative': 89, 'negative': 6011, 'positive': 89}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5776, 'false_negative': 324, 'negative': 5776, 'positive': 324}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5968, 'false_negative': 132, 'negative': 5968, 'positive': 132}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5065, 'false_negative': 1035, 'negative': 5065, 'positive': 1035}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5863, 'false_negative': 237, 'negative': 5863, 'positive': 237}, 'male': {'true_positive': 0, 'false_positive': 3, 'true_negative': 5151, 

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 6468, 'false_negative': 32, 'negative': 6468, 'positive': 32}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 6388, 'false_negative': 112, 'negative': 6388, 'positive': 112}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 6405, 'false_negative': 95, 'negative': 6405, 'positive': 95}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 6151, 'false_negative': 349, 'negative': 6151, 'positive': 349}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 6355, 'false_negative': 145, 'negative': 6355, 'positive': 145}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 5414, 'false_negative': 1086, 'negative': 5414, 'positive': 1086}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 6246, 'false_negative': 254, 'negative': 6246, 'positive': 254}, 'male': {'true_positive': 0, 'false_positive': 3, 'true_negative': 5500, 

In [23]:
print("------------------------")
for i in evaluation_by_annotation:
    print(i + ":")
    print(str(evaluation_by_annotation[i]))
print("------------------------")

------------------------
baby{'true_positive': 0, 'false_positive': 0, 'true_negative': 6539, 'false_negative': 32, 'negative': 6539, 'positive': 32}
bird{'true_positive': 0, 'false_positive': 0, 'true_negative': 6458, 'false_negative': 113, 'negative': 6458, 'positive': 113}
car{'true_positive': 0, 'false_positive': 0, 'true_negative': 6474, 'false_negative': 97, 'negative': 6474, 'positive': 97}
clouds{'true_positive': 0, 'false_positive': 0, 'true_negative': 6219, 'false_negative': 352, 'negative': 6219, 'positive': 352}
dog{'true_positive': 0, 'false_positive': 0, 'true_negative': 6425, 'false_negative': 146, 'negative': 6425, 'positive': 146}
female{'true_positive': 0, 'false_positive': 0, 'true_negative': 5462, 'false_negative': 1109, 'negative': 5462, 'positive': 1109}
flower{'true_positive': 0, 'false_positive': 0, 'true_negative': 6316, 'false_negative': 255, 'negative': 6316, 'positive': 255}
male{'true_positive': 0, 'false_positive': 3, 'true_negative': 5561, 'false_negative